**Software**

In [1]:
'''
Software
'''
import cv2
import numpy as np
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import time
import psutil

def apply_sepia(frame):
    # Aplicar el efecto sepia manualmente
    sepia_filter = np.array([[0.393, 0.769, 0.189],
                             [0.349, 0.686, 0.168],
                             [0.272, 0.534, 0.131]])
    sepia_frame = cv2.transform(frame, sepia_filter)
    sepia_frame = np.clip(sepia_frame, 0, 255).astype(np.uint8)
    return sepia_frame

# Función para medir la CPU y la memoria utilizada
def measure_resources():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Cargar el overlay base
base = BaseOverlay("base.bit")

# Configuración de la salida HDMI
hdmi_out = base.video.hdmi_out
hdmi_out.configure(VideoMode(640, 480, 24), PIXEL_BGR)
hdmi_out.start()

# Liberar cualquier recurso de cámara existente
cv2.VideoCapture(0).release()

# Inicializar la cámara
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Inicialización de variables para el cálculo de frames por segundo
start_time = time.time()
num_frames = 0
read_errors = 0
total_cpu_usage = 0
total_memory_usage = 0

while (base.buttons[3].read()==0):
    fstime=time.time()
    ret, frame = cap.read()
    if ret:
        # Incrementar el contador de frames
        num_frames += 1
        # Aplicar el filtro sepia
        sepia_frame = apply_sepia(frame)
        # Crear un nuevo objeto Frame
        frame_pynq = hdmi_out.newframe()
        # Copiar los datos del marco OpenCV al objeto Frame
        np.copyto(frame_pynq, sepia_frame)
        # Mostrar la imagen en el monitor HDMI Out
        cv2.putText(frame_pynq,"FPS:"+str(round(1/(time.time()-fstime),4)),(50,25),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(frame_pynq)
        
        # Medir recursos después de aplicar el filtro
        cpu_usage, memory_usage = measure_resources()
        total_cpu_usage += cpu_usage
        total_memory_usage += memory_usage
    else:
        # Incrementar el contador de errores de lectura
        read_errors += 1
        print("Error al leer de la cámara.")
            
# Calcular el tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time

# Calcular los frames por segundo
fps = (num_frames - read_errors) / elapsed_time

# Calcular el promedio de uso de CPU y memoria
avg_cpu_usage = total_cpu_usage / num_frames
avg_memory_usage = total_memory_usage / num_frames

# Mostrar resultados
print("Prueba finalizada")
print("Frames por segundo: " + str(fps))
print("Número de errores leídos: " + str(read_errors))
print("Uso promedio de CPU: " + str(avg_cpu_usage) + "%")
print("Uso promedio de memoria: " + str(avg_memory_usage) + "%")

# Liberar recursos
cap.release()
hdmi_out.stop()
hdmi_out.close()


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Prueba finalizada
Frames por segundo: 8.81224863848722
Número de errores leídos: 0
Uso promedio de CPU: 51.220338983050844%
Uso promedio de memoria: 59.22372881355928%


**Hardware**

In [2]:
'''
Hardware
'''
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import Overlay
import cv2
import os
import time
import psutil

# Función para medir la CPU y la memoria utilizada
def measure_resources():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

# Cargar el overlay
base = BaseOverlay("base_wrapper.bit")

# Configurar la salida HDMI
Mode = VideoMode(640, 480, 24)  # Modo del monitor: 640x480 @ 60Hz
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_BGR)
hdmi_out.start()

# Configuración de la cámara (entrada)
frame_in_w, frame_in_h = 640, 480

# Establecer nivel de registro de OpenCV en silencioso
os.environ["OPENCV_LOG_LEVEL"] = "SILENT"

# Liberar cualquier recurso de cámara existente
cv2.VideoCapture(0).release()

# Inicializar la cámara
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)

# Inicialización de variables para el cálculo de recursos
start_time = time.time()
num_frames = 0
read_errors = 0
total_cpu_usage = 0
total_memory_usage = 0

while (base.buttons[3].read()==0):
    # Capturar video frame por frame
    fstime=time.time()
    ret, frame_vga = videoIn.read()

    if ret:
        # Incrementar el contador de frames
        num_frames += 1

        # Aquí se aplica el procesamiento de imagen si es necesario
        processed_frame = frame_vga

        # Mostrar el frame procesado a través de HDMI Out
        outframe = hdmi_out.newframe()
        outframe[0:frame_in_h, 0:frame_in_w, :] = processed_frame
        cv2.putText(outframe,"FPS:"+str(round(1/(time.time()-fstime),4)),(50,25),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(outframe)

        # Medir recursos después de aplicar el filtro
        cpu_usage, memory_usage = measure_resources()
        total_cpu_usage += cpu_usage
        total_memory_usage += memory_usage
    else:
        # Incrementar el contador de errores de lectura
        read_errors += 1
        print("Error al leer de la cámara.")

# Calcular el tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time

# Calcular el promedio de uso de CPU y memoria
avg_cpu_usage = total_cpu_usage / num_frames
avg_memory_usage = total_memory_usage / num_frames

# Calcular los frames por segundo (FPS)
fps = num_frames / elapsed_time

# Mostrar resultados
print("Programa finalizado")
print("Frames por segundo: " + str(fps))
print("Número de errores leídos: " + str(read_errors))
print("Uso promedio de CPU: " + str(avg_cpu_usage) + "%")
print("Uso promedio de memoria: " + str(avg_memory_usage) + "%")

# Liberar recursos
videoIn.release()
hdmi_out.stop()
hdmi_out.close()


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Programa finalizado
Frames por segundo: 11.82716873086392
Número de errores leídos: 0
Uso promedio de CPU: 46.298611111111114%
Uso promedio de memoria: 68.58749999999999%
